In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import numpy as np

In [34]:
train_data = pd.read_csv('../data/train.csv', delimiter=',').drop(['Id'], axis=1)

(1460, 36)
(1460, 43)


In [31]:
non_number_data_description = categorical_columns.describe()
non_number_data_description

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


In [4]:
one_hot_categorical_columns = pd.get_dummies(categorical_columns)
one_hot_categorical_columns.sum(axis=0).head()

MSZoning_C (all)      10
MSZoning_FV           65
MSZoning_RH           16
MSZoning_RL         1151
MSZoning_RM          218
dtype: int64

In [28]:
def split_target_numerical_and_categorical_columns(df, target_name):
    target_column = train_data[target_name]
    numerical_columns = train_data.drop(columns=[target_name]).select_dtypes(include=[np.number])
    categorical_columns = train_data.drop(columns=[target_name]).select_dtypes(exclude=[np.number])
    return target_column, numerical_columns, categorical_columns

In [ ]:
def one_hot_encode_categorical_data(categorical_data):
    one_hot_encoded = pd.get_dummies(categorical_data)

In [35]:
target_column, numerical_columns, categorical_columns = split_target_numerical_and_categorical_columns(train_data, 'SalePrice')
print(target_column.shape)
print(numerical_columns.shape)
print(categorical_columns.shape)

(1460,)
(1460, 36)
(1460, 43)


In [22]:
def filter_columns_corr_more_than_x(x, df, target_series, target_name):
    df_and_target = pd.concat([df, train_data[target_name]], axis=1)
    df_and_target_corr = df_and_target.corr().abs().drop(columns=[target_name])
    return df.loc[:, (df_and_target_corr.loc[target_name,:]>=x)]

In [26]:
train_cat_data = filter_columns_corr_more_than_x(0.3, one_hot_categorical_columns, train_data['SalePrice'], 'SalePrice')
num_cat_columns = train_cat_data.columns
print(train_cat_data.shape)
train_num_data = filter_columns_corr_more_than_x(0.3, numerical_columns, train_data['SalePrice'], 'SalePrice')
num_data_columns = train_num_data.columns
print(train_num_data.shape)

(1460, 27)
(1460, 18)
